In [34]:
import operator 
import json
import nltk
import re
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from collections import Counter
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk import bigrams

#HashTag to Check
hashtag = 'brexit'

[nltk_data] Downloading package punkt to /Users/cesar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/cesar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

In [16]:
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [49]:
# Remove stop-words (common words)  NLTK provides a simple list for English stop-words
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['RT', 'via', '#Brexit','…','https',':/']

In [50]:
# Tokenizer and counter
with open(hashtag+'.json', 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]
        # Update the counter
        count_all.update(terms_stop)
    # Print the first 5 most frequent words
    print(count_all.most_common(15))

[('The', 132), ('#brexit', 131), ('blue', 100), ('passports', 94), ('government', 84), ('statement', 83), ('issued', 81), ('reintroduction', 81), ('https://t.co/ny8KdoW2oK', 81), ('@Scarfolk', 80), ('EU', 65), ('amp', 61), ('war', 55), ('Spain', 53), ('UK', 44)]


In [51]:
# The bigrams() function from NLTK will take a list of tokens and 
# produce a list of tuples using adjacent tokens
terms_bigram = bigrams(terms_stop)
count_bi = Counter()
count_bi.update(terms_bigram)
count_bi.most_common(20)

[(('@FabianPicardo', 'My'), 1),
 (('My', 'article'), 1),
 (('article', "today's"), 1),
 (("today's", '@Daily_Express'), 1),
 (('@Daily_Express', '#Gibraltar'), 1),
 (('#Gibraltar', 'amp'), 1),
 (('amp', 'https://t.co/x5LjAnSHae'), 1),
 (('https://t.co/x5LjAnSHae', 'https://t.co/HeJeJ2XaGS'), 1)]